In [12]:
import os
import pandas as pd
list_ = os.listdir('sorted_data/128/Trajectory/')
sorted_41 = []
for i in range(len(list_)):
    file_ = open('sorted_data/128/Trajectory/' + list_[i], 'r') 
    file = file_.readlines()
    file = file[6:]
    for j in range(len(file)):
        tmp = file[j].split(',')
        sorted_41.append([tmp[0], tmp[1], tmp[-2] + ' ' +tmp[-1][:-1]])

df = pd.DataFrame(sorted_41)
df[2] = pd.to_datetime(df[2])
df.sort_values(by = [2],inplace=True)
df = df.reset_index()
df = df[[0,1,2]]

In [1]:
import random
import numpy as np
import torch

# multivariate data preparation
from numpy import array
from numpy import hstack
 
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps):
    X, y = list(), list()
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the dataset
        if end_ix > len(sequences):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1, -1]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)
 
# define input sequence
in_seq1 = array([x for x in range(0,100,10)])
in_seq2 = array([x for x in range(5,105,10)])
out_seq = array([in_seq1[i]+in_seq2[i] for i in range(len(in_seq1))])
# convert to [rows, columns] structure
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
out_seq = out_seq.reshape((len(out_seq), 1))
# horizontally stack columns
dataset = hstack((in_seq1, in_seq2, out_seq))

In [17]:
out_seq

array([[  5],
       [ 25],
       [ 45],
       [ 65],
       [ 85],
       [105],
       [125],
       [145],
       [165],
       [185]])

In [14]:
df[0], df[1]

(0                  40.07585
 1          40.0758333333333
 2                    40.076
 3                   40.0763
 4                   40.0766
                  ...       
 1208495          39.9756416
 1208496            39.97564
 1208497          39.9756233
 1208498          39.9755883
 1208499            39.97557
 Name: 0, Length: 1208500, dtype: object,
 0          116.327816666667
 1          116.327583333333
 2          116.327566666667
 3                  116.3275
 4          116.327416666667
                  ...       
 1208495         116.3302849
 1208496         116.3302566
 1208497         116.3302866
 1208498         116.3303116
 1208499           116.33036
 Name: 1, Length: 1208500, dtype: object)

In [9]:
class MV_LSTM(torch.nn.Module):
    def __init__(self,n_features,seq_length):
        super(MV_LSTM, self).__init__()
        self.n_features = n_features
        self.seq_len = seq_length
        self.n_hidden = 20 # number of hidden states
        self.n_layers = 1 # number of LSTM layers (stacked)
    
        self.l_lstm = torch.nn.LSTM(input_size = n_features, 
                                 hidden_size = self.n_hidden,
                                 num_layers = self.n_layers, 
                                 batch_first = True)
        # according to pytorch docs LSTM output is 
        # (batch_size,seq_len, num_directions * hidden_size)
        # when considering batch_first = True
        self.l_linear = torch.nn.Linear(self.n_hidden*self.seq_len, 1)
        
    
    def init_hidden(self, batch_size):
        # even with batch_first = True this remains same as docs
        hidden_state = torch.zeros(self.n_layers,batch_size,self.n_hidden)
        cell_state = torch.zeros(self.n_layers,batch_size,self.n_hidden)
        self.hidden = (hidden_state, cell_state)
    
    
    def forward(self, x):
        batch_size, seq_len, _ = x.size()
        
        lstm_out, self.hidden = self.l_lstm(x,self.hidden)
        # lstm_out(with batch_first = True) is 
        # (batch_size,seq_len,num_directions * hidden_size)
        # for following linear layer we want to keep batch_size dimension and merge rest       
        # .contiguous() -> solves tensor compatibility error
        x = lstm_out.contiguous().view(batch_size,-1)
        return self.l_linear(x)

In [25]:
n_features = 1 # this is number of parallel inputs
n_timesteps = 3 # this is number of timesteps

# convert dataset into input/output
X, y = split_sequences(dataset, n_timesteps)
y = np.vstack([y,y])
print(X.shape, y.shape)

# create NN
mv_net_x = MV_LSTM(n_features,n_timesteps)
mv_net_y = MV_LSTM(n_features, n_timesteps)
criterion = torch.nn.MSELoss() # reduction='sum' created huge loss value
optimizer_x = torch.optim.Adam(mv_net_x.parameters(), lr=1e-1)
optimizer_y = torch.optim.Adam(mv_net_y.parameters(), lr=1e-1)

train_episodes = 500
batch_size = 16

(8, 3, 2) (2, 8)


In [26]:
X

array([[[ 0,  5],
        [10, 15],
        [20, 25]],

       [[10, 15],
        [20, 25],
        [30, 35]],

       [[20, 25],
        [30, 35],
        [40, 45]],

       [[30, 35],
        [40, 45],
        [50, 55]],

       [[40, 45],
        [50, 55],
        [60, 65]],

       [[50, 55],
        [60, 65],
        [70, 75]],

       [[60, 65],
        [70, 75],
        [80, 85]],

       [[70, 75],
        [80, 85],
        [90, 95]]])

In [24]:
type(X)

numpy.ndarray

In [27]:
mv_net_x.train()
mv_net_y.train()

for t in range(train_episodes):
    for b in range(0,len(X),batch_size):
        inpt = X[b:b+batch_size,:,:]
        target = y[b:b+batch_size]    
        
        x_batch = torch.tensor(inpt[:,:,0],dtype=torch.float32)
        y_batch = torch.tensor(inpt[:,:,1],dtype=torch.float32)
        target_batch = torch.tensor(target,dtype=torch.float32)
    
        mv_net_x.init_hidden(x_batch.size(0))
        mv_net_y.init_hidden(y_batch.size(0))
    #    lstm_out, _ = mv_net.l_lstm(x_batch,nnet.hidden)    
    #    lstm_out.contiguous().view(x_batch.size(0),-1)
        x_batch = x_batch.reshape(8,3,1) # 3 채널로 맞춰줘야 함
        y_batch = y_batch.reshape(8,3,1)
        output_x = mv_net_x(x_batch)
        output_y = mv_net_y(y_batch)
        loss = criterion(output_x.view(-1), target_batch[0,:]) + criterion(output_y.view(-1), target_batch[1,:])
        
        loss.backward()
        optimizer_x.step()        
        optimizer_x.zero_grad()
        optimizer_y.step()
        optimizer_y.zero_grad()
    print('step : ' , t , 'loss : ' , loss.item())

step :  0 loss :  30562.6953125
step :  1 loss :  29500.064453125
step :  2 loss :  27658.447265625
step :  3 loss :  26146.0703125
step :  4 loss :  24701.228515625
step :  5 loss :  23122.08203125
step :  6 loss :  21718.173828125
step :  7 loss :  20306.89453125
step :  8 loss :  18937.60546875
step :  9 loss :  17651.4921875
step :  10 loss :  16332.0771484375
step :  11 loss :  15099.9267578125
step :  12 loss :  13888.5546875
step :  13 loss :  12799.01171875
step :  14 loss :  11760.1220703125
step :  15 loss :  10694.171875
step :  16 loss :  9778.779296875
step :  17 loss :  8992.82421875
step :  18 loss :  8275.5146484375
step :  19 loss :  7623.88916015625
step :  20 loss :  7036.7177734375
step :  21 loss :  6511.962890625
step :  22 loss :  6046.8857421875
step :  23 loss :  5638.181640625
step :  24 loss :  5282.0966796875
step :  25 loss :  4974.5810546875
step :  26 loss :  4711.4248046875
step :  27 loss :  4488.4072265625
step :  28 loss :  4301.4375
step :  29 loss :

step :  260 loss :  1.105055809020996
step :  261 loss :  0.7696412801742554
step :  262 loss :  0.3095557689666748
step :  263 loss :  0.150666281580925
step :  264 loss :  0.36970430612564087
step :  265 loss :  0.615480899810791
step :  266 loss :  0.5441474318504333
step :  267 loss :  0.2917678952217102
step :  268 loss :  0.11669367551803589
step :  269 loss :  0.171170175075531
step :  270 loss :  0.33931970596313477
step :  271 loss :  0.4009891450405121
step :  272 loss :  0.3254457712173462
step :  273 loss :  0.16989833116531372
step :  274 loss :  0.08079826086759567
step :  275 loss :  0.10187168419361115
step :  276 loss :  0.18404985964298248
step :  277 loss :  0.24741655588150024
step :  278 loss :  0.22404217720031738
step :  279 loss :  0.15563467144966125
step :  280 loss :  0.07973581552505493
step :  281 loss :  0.05260876193642616
step :  282 loss :  0.07962819933891296
step :  283 loss :  0.12275317311286926
step :  284 loss :  0.14706040918827057
step :  285 lo

step :  482 loss :  0.28119802474975586
step :  483 loss :  0.35296791791915894
step :  484 loss :  0.43652892112731934
step :  485 loss :  0.5086459517478943
step :  486 loss :  0.5784029960632324
step :  487 loss :  0.5929780006408691
step :  488 loss :  0.5842443704605103
step :  489 loss :  0.49188223481178284
step :  490 loss :  0.38907885551452637
step :  491 loss :  0.2549743354320526
step :  492 loss :  0.14870238304138184
step :  493 loss :  0.06784426420927048
step :  494 loss :  0.02274903655052185
step :  495 loss :  0.007861128076910973
step :  496 loss :  0.01680678129196167
step :  497 loss :  0.04274555668234825
step :  498 loss :  0.08109237253665924
step :  499 loss :  0.13029395043849945
